In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
    ------
    {context}
    """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {
                "context": doc.page_content,
                "question": question,
            }
        ).content
        for doc in documents
    )


map_chain = {"documents": retriver, "question": RunnablePassthrough()} | RunnableLambda(
    map_docs
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        "system",
        """
    Given the following extracted parts of a long document and a question, create a final answer. If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    ------
    {context}
    """,
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()
         } | final_prompt | llm

chain.invoke("what is AXI?")

AIMessage(content='AXI stands for Advanced eXtensible Interface. It is a widely used bus protocol that provides a high-performance and low-latency interface between different components in a system-on-chip (SoC) design. AXI facilitates efficient data transfer and communication between processors, memory controllers, and other peripherals in a system. It supports features such as burst transfers, multiple data widths, and quality of service (QoS) signaling to optimize performance and resource utilization.')

In [47]:
# list of docs

# for doc in list of docs | prompt | llm

# for response in list of llm response | put them all together

# final doc | prompt | llm

# chain.invoke(question)

[Document(page_content='Central Interconnect\nThe central interconnect is the core of the Arm NIC301-based interconnect switches.\nMaster Interconnect\nThe master interconnect switches the low-to-medium speed traffic from the central interconnect to M_AXI_GP ports, I/O peripherals (IOP) and other blocks.\nSlave Interconnect\nThe slave interconnect switches the low-to-medium speed traffic from S_AXI_GP ports, DevC and DAP to the central interconnect.\nMemory Interconnect\nThe memory interconnect switches the high speed traffic from the AXI_HP ports to DDR DRAM and on-chip RAM (through another interconnect).\nOCM Interconnect\nThe OCM interconnect switches the high speed traffic from the central interconnect and the memory interconnect.\nL2 Cache Controller\nThe functionality of the L2 cache controller is described in Application Processing Unit. The address filtering feature of the L2 cache controller makes the L2 cache controller function like a switch from the perspective of the traff

In [3]:
from langchain.llms import GPT4All
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "You are a helpful assistant that defines words. Define this word: {word}.")

llm = GPT4All(
    model="./gpt4all-falcon-q4_0.gguf",
)

chain = prompt | llm

chain.invoke({"word": "potato"})

' root vegetable that is native to South America. It is a staple food in many countries around the world, including Peru, Bolivia, and Colombia. Potatoes are easy to grow and can be harvested year-round. They are also versatile and can be used in a variety of dishes, from mashed potatoes to baked potatoes.'